Capstone Project

Dwanith Venkat Girish

Import all Packages/libraries

In [ ]:

import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import sys

Import libraries from RL-Pruner code

In [ ]:

sys.path.append('/content/drive/My Drive/Capstone/RLPruner_CNN')
!ls -d "/content/drive/My Drive/Capstone/RLPruner_CNN"/*/

Mounted at /content/drive
'/content/drive/My Drive/Capstone/RLPruner_CNN/assets/'
'/content/drive/My Drive/Capstone/RLPruner_CNN/conf/'
'/content/drive/My Drive/Capstone/RLPruner_CNN/models/'
'/content/drive/My Drive/Capstone/RLPruner_CNN/pretrained_model/'
'/content/drive/My Drive/Capstone/RLPruner_CNN/RLPruner-CNN/'
'/content/drive/My Drive/Capstone/RLPruner_CNN/scripts/'
'/content/drive/My Drive/Capstone/RLPruner_CNN/utils/'


In [ ]:

#Installing requirements.txt (Requirements of the paper)
#Only run once -> restart session (automatic) -> run the cell up and below this only


!pip install -r "/content/drive/My Drive/Capstone/RLPruner_CNN/requirements.txt"



!pip install thop


Phase 1: ResNet18 model training, model saved in pretrained_model.py

In [ ]:


!python "/content/drive/My Drive/Capstone/RLPruner_CNN/RLPruner-CNN/train.py" \
        --model resnet18 \
        --dataset cifar100 \
        --device cuda


python3: can't open file '/content/drive/My Drive/Capstone/RLPruner_CNN/RLPruner-CNN/train.py': [Errno 2] No such file or directory


Phase 2: Resnet 34, 50, 101, 152.

In [ ]:
!python "/content/drive/My Drive/Capstone/RLPruner_CNN/RLPruner-CNN/train.py" \
        --model resnet34 \
        --dataset cifar100 \
        --device cuda


Start with random seed: 1
Files already downloaded and verified
Files already downloaded and verified
Training: 100% 250/250 [43:08<00:00, 10.35s/epoch, Train loss=0.00664, Best top1 acc=0.778, Top1 acc=0.775]
Pretrained model saved at pretrained_model/resnet34_cifar100_pretrained.pth


In [ ]:
#Resnet 50

!python "/content/drive/My Drive/Capstone/RLPruner_CNN/RLPruner-CNN/train.py" \
        --model resnet50 \
        --dataset cifar100 \
        --device cuda


wandb: WARNING Tried to auto resume run with id 1745609914 but id 1745614065 is set.
Start with random seed: 1
Files already downloaded and verified
Files already downloaded and verified
Training: 100% 250/250 [1:21:33<00:00, 19.58s/epoch, Train loss=0.00725, Best top1 acc=0.793, Top1 acc=0.791]
Pretrained model saved at pretrained_model/resnet50_cifar100_pretrained.pth


In [ ]:
#Resnet101

!python "/content/drive/My Drive/Capstone/RLPruner_CNN/RLPruner-CNN/train.py" \
        --model resnet101 \
        --dataset cifar100 \
        --device cuda


Start with random seed: 1
100% 169M/169M [00:01<00:00, 91.8MB/s]
Training: 100% 250/250 [2:10:38<00:00, 31.35s/epoch, Train loss=0.00623, Best top1 acc=0.802, Top1 acc=0.8]
Pretrained model saved at pretrained_model/resnet101_cifar100_pretrained.pth


In [ ]:
#Resnet152


!python "/content/drive/My Drive/Capstone/RLPruner_CNN/RLPruner-CNN/train.py" \
        --model resnet152 \
        --dataset cifar100 \
        --device cuda


wandb: WARNING Ignoring ID 1745812671 loaded due to resume='auto' because the run ID is set to 1745822773.
Start with random seed: 1
Training: 100% 250/250 [3:04:28<00:00, 44.28s/epoch, Train loss=0.00653, Best top1 acc=0.802, Top1 acc=0.802]
Pretrained model saved at pretrained_model/resnet152_cifar100_pretrained.pth


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from models import resnet
import torch.nn.functional as F


model_path = "/content/drive/My Drive/Capstone/RLPruner_CNN/pretrained_model/resnet18_cifar100_pretrained.pth"
model = torch.load(model_path, weights_only=False).cuda()
model.eval()


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)


correct, total = 0, 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Top-1 Accuracy: {100 * correct / total:.2f}%")


ModuleNotFoundError: No module named 'models'

Performance metrics of Pre-trained ResNet18

Compressing RESNET18 (RLPruner)

In [ ]:
!python "/content/drive/My Drive/Capstone/RLPruner_CNN/compress.py" \
        --model resnet18 \
        --dataset cifar100 \
        --device cuda \
        --prune_filter_ratio 0.2 \
        --prune_strategy taylor \
        --Q_FLOP_coef 0.2 \
        --Q_Para_coef 0.0 \
        --pretrained_pth "/content/drive/My Drive/Capstone/RLPruner_CNN/pretrained_model/resnet18_cifar100_pretrained.pth"


python3: can't open file '/content/drive/My Drive/Capstone/RLPruner_CNN/compress.py': [Errno 2] No such file or directory


L1-Norm pruning method

In [ ]:


def l1_prune(model, prune_ratio=0.2):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            weight = module.weight.data.abs().clone()
            l1_norm = weight.view(weight.size(0), -1).sum(dim=1)
            num_prune = int(prune_ratio * l1_norm.size(0))
            if num_prune == 0:
                continue
            prune_indices = l1_norm.argsort()[:num_prune]
            module.weight.data[prune_indices] = 0



transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test_set = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=128, shuffle=False, num_workers=2)



def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100 * correct / total

model_path = "/content/drive/My Drive/Capstone/RLPruner_CNN/pretrained_model/resnet18_cifar100_pretrained.pth"
model = torch.load(model_path, weights_only=False).cuda()
model.eval()


l1_prune(model, prune_ratio=0.10)
test_accuracy = evaluate(model, test_loader)
print(f"Pruned Model Accuracy: {test_accuracy:.2f}%")


Pruned Model Accuracy: 50.51%


Resnet18 (fine-tuning) CIFAR100

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_set = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=128, shuffle=True, num_workers=2)



criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=5e-4, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

model.train()
for epoch in range(40):  # fine-tune for 40 epochs
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.3f}")

test_accuracy = evaluate(model, test_loader)
print(f"Fine-tuned Accuracy: {test_accuracy:.2f}%")



Epoch 1, Loss: 1811.854
Epoch 2, Loss: 1626.337
Epoch 3, Loss: 1598.770
Epoch 4, Loss: 1595.346
Epoch 5, Loss: 1595.253
Epoch 6, Loss: 1550.093
Epoch 7, Loss: 1544.336
Epoch 8, Loss: 1539.268
Epoch 9, Loss: 1543.034
Epoch 10, Loss: 1538.996
Epoch 11, Loss: 1512.370
Epoch 12, Loss: 1506.753
Epoch 13, Loss: 1495.103
Epoch 14, Loss: 1487.314
Epoch 15, Loss: 1482.535
Epoch 16, Loss: 1447.878
Epoch 17, Loss: 1428.705
Epoch 18, Loss: 1404.370
Epoch 19, Loss: 1380.850
Epoch 20, Loss: 1351.202
Epoch 21, Loss: 1290.638
Epoch 22, Loss: 1265.792
Epoch 23, Loss: 1244.825
Epoch 24, Loss: 1230.174
Epoch 25, Loss: 1210.530
Epoch 26, Loss: 1162.116
Epoch 27, Loss: 1143.409
Epoch 28, Loss: 1129.547
Epoch 29, Loss: 1113.004
Epoch 30, Loss: 1104.940
Epoch 31, Loss: 1066.700
Epoch 32, Loss: 1054.670
Epoch 33, Loss: 1042.835
Epoch 34, Loss: 1029.963
Epoch 35, Loss: 1021.312
Epoch 36, Loss: 992.973
Epoch 37, Loss: 982.644
Epoch 38, Loss: 973.665
Epoch 39, Loss: 964.746
Epoch 40, Loss: 953.119
Fine-tuned Acc